In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Sitka,57.0531,-135.3300,46.29,71,90,11.50,US,2021-11-05 16:39:11
1,1,Presidencia Roque Saenz Pena,-26.7852,-60.4388,57.04,95,100,14.72,AR,2021-11-05 16:39:11
2,2,San Jeronimo,-11.9472,-75.2829,61.36,53,61,6.62,PE,2021-11-05 16:39:12
3,3,Mataura,-46.1927,168.8643,40.48,95,89,2.57,NZ,2021-11-05 16:39:12
4,4,Bengkulu,-3.8004,102.2655,76.28,91,100,5.86,ID,2021-11-05 16:39:12


In [2]:
# GET THE DATA TYPES
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig 


Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,Bengkulu,-3.8004,102.2655,76.28,91,100,5.86,ID,2021-11-05 16:39:12
8,8,Gunjur,13.2019,-16.7339,89.67,51,40,10.36,GM,2021-11-05 16:39:14
9,9,Garissa,-0.4536,39.6401,88.93,41,41,15.59,KE,2021-11-05 16:39:14
12,12,Mahebourg,-20.4081,57.7000,75.49,88,40,3.44,MU,2021-11-05 16:39:15
13,13,Cabo San Lucas,22.8909,-109.9124,83.05,58,52,1.99,MX,2021-11-05 16:36:45
15,15,Victoria,22.2855,114.1577,77.38,87,21,1.99,HK,2021-11-05 16:39:16
19,19,Caravelas,-17.7125,-39.2481,81.12,68,78,9.40,BR,2021-11-05 16:39:17
33,33,Honiara,-9.4333,159.9500,77.65,83,97,0.45,SB,2021-11-05 16:39:21
38,38,Georgetown,5.4112,100.3354,78.89,92,20,3.00,MY,2021-11-05 16:39:23
40,40,Arraial Do Cabo,-22.9661,-42.0278,78.76,69,0,16.11,BR,2021-11-05 16:39:23


In [10]:
preferred_cities_df.count()

City_ID       161
City          161
Lat           161
Lng           161
Max Temp      161
Humidity      161
Cloudiness    161
Wind Speed    161
Country       161
Date          161
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Bengkulu,ID,76.28,-3.8004,102.2655,
8,Gunjur,GM,89.67,13.2019,-16.7339,
9,Garissa,KE,88.93,-0.4536,39.6401,
12,Mahebourg,MU,75.49,-20.4081,57.7000,
13,Cabo San Lucas,MX,83.05,22.8909,-109.9124,
15,Victoria,HK,77.38,22.2855,114.1577,
19,Caravelas,BR,81.12,-17.7125,-39.2481,
33,Honiara,SB,77.65,-9.4333,159.9500,
38,Georgetown,MY,78.89,5.4112,100.3354,
40,Arraial Do Cabo,BR,78.76,-22.9661,-42.0278,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [17]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))